# Fusion Retrieval을 활용한 문서 검색

## 개요

이 코드는 벡터 기반의 유사도 검색과 키워드 기반의 BM25 검색을 결합한 Fusion Retrieval 시스템을 구현합니다. 이 접근 방식은 두 방법의 강점을 활용하여 문서 검색의 전반적인 품질과 관련성을 향상시키는 것을 목표로 합니다.

## 동기

전통적인 검색 방법은 주로 의미적 이해(벡터 기반) 또는 키워드 매칭(BM25)에 의존합니다. 각 접근 방식에는 장단점이 존재하며, Fusion Retrieval은 이러한 방법들을 결합하여 보다 견고하고 정확한 검색 시스템을 구축하려는 시도입니다. 이를 통해 다양한 유형의 쿼리를 효과적으로 처리할 수 있습니다.

## 주요 구성 요소

1. PDF 처리 및 텍스트 청크 분할
2. FAISS와 OpenAI 임베딩을 활용한 벡터 스토어 생성
3. 키워드 기반 검색을 위한 BM25 인덱스 생성
4. 두 가지 방법을 결합하는 커스텀 Fusion Retrieval 함수

## 방법 상세 설명

### 문서 전처리

1. PDF 파일을 로드하고 RecursiveCharacterTextSplitter를 사용해 텍스트를 청크로 분할합니다.
2. 텍스트 청크의 형식 문제를 해결하기 위해 't'를 공백으로 교체하여 청크를 정리합니다.

### 벡터 스토어 생성

1. OpenAI 임베딩을 사용해 텍스트 청크의 벡터 표현을 생성합니다.
2. 생성된 임베딩을 활용하여 FAISS 벡터 스토어를 생성하고 효율적인 유사도 검색을 수행합니다.

### BM25 인덱스 생성

1. 벡터 스토어에 사용된 동일한 텍스트 청크를 바탕으로 BM25 인덱스를 생성합니다.
2. 이를 통해 벡터 기반 검색과 함께 키워드 기반 검색을 수행할 수 있습니다.

### Fusion Retrieval 함수

`fusion_retrieval` 함수는 이 구현의 핵심입니다:

1. 쿼리를 입력받아 벡터 기반과 BM25 기반 검색을 모두 수행합니다.
2. 두 검색 방법에서 얻은 점수를 공통된 척도로 정규화합니다.
3. 이러한 점수들을 가중치(`alpha` 파라미터)를 사용하여 결합합니다.
4. 결합된 점수를 기반으로 문서들을 랭킹하고, 상위 k개의 결과를 반환합니다.

## 접근 방식의 장점

1. **향상된 검색 품질**: 의미와 키워드 기반 검색을 결합함으로써 개념적 유사성과 정확한 키워드 매칭을 모두 포착할 수 있습니다.
2. **유연성**: `alpha` 파라미터를 조정하여 벡터와 키워드 검색의 균형을 사용 사례나 쿼리 유형에 맞게 조절할 수 있습니다.
3. **견고성**: 결합된 접근 방식은 개별 방법의 약점을 보완하며 다양한 쿼리에 효과적으로 대응할 수 있습니다.
4. **사용자 정의 가능성**: 이 시스템은 다른 벡터 스토어나 키워드 기반 검색 방법을 쉽게 적용할 수 있어 적응성이 높습니다.

## 결론

Fusion Retrieval은 의미적 이해와 키워드 매칭의 강점을 결합한 강력한 문서 검색 접근 방식입니다. 벡터 기반과 BM25 검색 방법을 모두 활용함으로써 정보 검색 작업에 대해 더 포괄적이고 유연한 솔루션을 제공합니다. 이 접근 방식은 학술 연구, 법률 문서 검색, 일반적인 검색 엔진 등 개념적 유사성과 키워드 관련성이 중요한 다양한 분야에서 활용 가능성이 있습니다.


<div style="text-align: center;">

<img src="../images/fusion_retrieval.svg" alt="Fusion Retrieval" style="width:100%; height:auto;">
</div>

### Import libraries 

In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain.docstore.document import Document

from typing import List
from rank_bm25 import BM25Okapi
import numpy as np


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_4345/3231833842.py:12: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import *
/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.5, however version 1.4.7 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


### Define document path

In [2]:
path = "../data/Understanding_Climate_Change.pdf"

### Encode the pdf to vector store and return split document from the step before to create BM25 instance
### pdf를 벡터 스토어에 인코딩하고 문서를 분리하여 반환함. 

In [4]:
def encode_pdf_and_get_split_documents(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore, cleaned_texts

### 청킹된 문서를 임베딩하여 벡터스토어에 넣음 

In [5]:
vectorstore, cleaned_texts = encode_pdf_and_get_split_documents(path)

In [6]:
cleaned_texts

[Document(metadata={'source': '../data/Understanding_Climate_Change.pdf', 'page': 0}, page_content='Understanding Climate Change  \nChapter 1: Introduction to Climate Change  \nClimate change refers to significant, long -term changes in the global climate. The term \n"global climate" encompasses the planet\'s overall weather patterns, including temperature, \nprecipitation, and wind patterns, over an extended period. Over the past cent ury, human \nactivities, particularly the burning of fossil fuels and deforestation, have significantly \ncontributed to climate change.  \nHistorical Context  \nThe Earth\'s climate has changed throughout history. Over the past 650,000 years, there have \nbeen seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about \n11,700 years ago marking the beginning of the modern climate era and  human civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount 

In [ ]:
[doc.page_content.split() for doc in documents]

### Create a bm25 index for retrieving documents by keywords
- BM25는 TF-IDF와 유사하게 단어 빈도와 문서 내 단어의 희소성을 고려하지만, 두 가지 주요 개선점을 가지고 있습니다.

1. 단어 빈도의 포화: TF-IDF에서는 단어가 문서에 많이 등장할수록 점수가 계속 증가합니다. 그러나 BM25에서는 빈도가 일정 수준 이상 높아지면 점수를 증가시키지 않습니다.
2. 문서 길이 보정: BM25는 문서의 길이를 고려하여 짧은 문서와 긴 문서를 공정하게 평가합니다.

In [9]:
def create_bm25_index(documents: List[Document]) -> BM25Okapi:
    """
    주어진 문서들로부터 BM25 인덱스를 생성합니다.

    BM25 (Best Matching 25)는 정보 검색에서 사용되는 순위 함수입니다.
    이는 확률적 검색 프레임워크를 기반으로 하며 TF-IDF의 개선된 버전입니다.

    Args:
    documents (List[Document]): 인덱싱할 문서들의 리스트.

    Returns:
    BM25Okapi: BM25 점수를 계산하는 데 사용할 수 있는 인덱스.
    """
    # 각 문서를 공백을 기준으로 토크나이즈하여 나눕니다.
    # 이는 간단한 접근 방식이며, 더 정교한 토크나이저를 사용하여 개선할 수 있습니다.
    tokenized_docs = [doc.page_content.split() for doc in documents]

    # BM25Okapi 인덱스를 반환하여 이후 검색에 사용할 수 있도록 합니다.
    return BM25Okapi(tokenized_docs)


In [10]:
bm25 = create_bm25_index(cleaned_texts) # Create BM25 index from the cleaned texts (chunks)

### Define a function that retrieves both semantically and by keyword, normalizes the scores and gets the top k documents

In [13]:
def fusion_retrieval(vectorstore, bm25, query: str, k: int = 5, alpha: float = 0.5) -> List[Document]:
    """
    키워드 기반(BM25) 및 벡터 기반 검색을 결합하여 Fusion Retrieval을 수행합니다.

    Args:
    vectorstore (VectorStore): 벡터 스토어(FAISS) 인스턴스로 문서가 포함된 스토어.
    bm25 (BM25Okapi): 사전에 계산된 BM25 인덱스.
    query (str): 사용자 입력 쿼리 문자열.
    k (int): 반환할 문서 개수.
    alpha (float): 벡터 검색 점수의 가중치 (BM25 점수의 가중치는 1-alpha로 설정).

    Returns:
    List[Document]: 결합 점수를 기준으로 상위 k개의 문서.
    """
    
    # Step 1: 벡터 스토어에서 모든 문서 가져오기
    # 모든 문서를 가져와야 결합 검색을 위해 BM25 점수와 벡터 검색 점수를 결합할 수 있음
    all_docs = vectorstore.similarity_search("", k=vectorstore.index.ntotal)

    # Step 2: BM25 검색 수행
    # BM25 인덱스를 사용하여 쿼리에 대한 키워드 기반 검색 점수를 계산
    bm25_scores = bm25.get_scores(query.split())

    # Step 3: 벡터 검색 수행
    # 벡터 스토어에서 쿼리와 유사도가 높은 문서들의 점수를 가져옴
    vector_results = vectorstore.similarity_search_with_score(query, k=len(all_docs))
    
    # Step 4: 점수 정규화
    # 벡터 점수는 min-max sclaer 통해 0과 1 사이로 조정됨
    vector_scores = np.array([score for _, score in vector_results])
    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores))

    # BM25 점수도 동일하게 min-max sclaer 통해 0과 1 사이로 조정
    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))

    # Step 5: 점수 결합
    # 벡터 점수와 BM25 점수를 결합하여 최종 점수를 계산
    # alpha는 벡터 점수의 가중치, (1 - alpha)는 BM25 점수의 가중치
    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores  

    # Step 6: 문서 순위 정렬
    # 결합된 점수를 기준으로 문서의 인덱스를 내림차순 정렬하여 상위 점수 문서부터 가져옴
    sorted_indices = np.argsort(combined_scores)[::-1]
    
    # Step 7: 상위 k개의 문서 반환
    # 정렬된 인덱스를 통해 상위 k개의 문서를 반환
    return [all_docs[i] for i in sorted_indices[:k]]


### Use Case example

In [14]:
# Query
query = "What are the impacts of climate change on the environment?"

# Perform fusion retrieval
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.5)
docs_content = [doc.page_content for doc in top_docs]
show_context(docs_content)

Context 1:
workshops, and community events. Lifelong learning fosters a culture of con tinuous 
improvement and adaptability.  
Intergenerational Dialogue  
Youth Engagement  
Engaging youth in climate action is critical for long -term sustainability. Youth bring energy, 
creativity, and a sense of urgency to climate movements. Providing platforms for youth 
voices, supporting youth -led initiatives, and involving young people in de cision -making 
processes are essential for meaningful engagement.  
Intergenerational Collaboration  
Intergenerational collaboration involves working together across age groups to address 
climate challenges. This includes mentorship programs, intergenerational projects, and 
dialogue forums. Sharing knowledge and experiences between generations enhances 
collective capacity and resilience.


Context 2:
This vision includes a healthy planet, thriving ecosystems, and equitable societies. Working 
together towards this vision creates a sense of purpose and 

### 내 생각 
- 유사도를 구하는 방식을 두 가지로 나누어 계산
- 계산한 값들을 가중치를 활용하여 재계산한 후 재랭킹하여 top-k만큼 뽑아낸다. 